<a href="https://colab.research.google.com/github/eyang16/ECGR_4105_ML_Fall_2025/blob/main/Homework_7/ECGR_4105_ML_HW7_Eyang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Homework 7
# Ethan Yang
# ID: 801250277
# ECGR_4105_Machine_Learning

In [ ]:
import torch
import subprocess
import platform

print("PyTorch reports CUDA available:", torch.cuda.is_available())

# If CUDA is available, print GPU name
if torch.cuda.is_available():
    print("PyTorch CUDA device:", torch.cuda.get_device_name(0))
else:
    print("No CUDA device detected by PyTorch.")

# Also check using system tools:
print("\n=== System GPU Check ===")

system = platform.system()

if system == "Windows":
    try:
        output = subprocess.check_output("wmic path win32_VideoController get name", shell=True)
        print(output.decode())
    except:
        print("Could not query GPU on Windows.")
elif system == "Linux":
    try:
        output = subprocess.check_output("lspci | grep -i vga", shell=True)
        print(output.decode())
    except:
        print("Could not query GPU on Linux.")
else:
    print("Unsupported OS for GPU check.")


PyTorch reports CUDA available: True
PyTorch CUDA device: Tesla T4

=== System GPU Check ===
Could not query GPU on Linux.


In [ ]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
# Problem 1.a
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Data Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

# CNN Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)  # 10 classes for CIFAR-10
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

net = Net()

# Training Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training Loop
epochs = 300
start_time = time.time()

for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

training_time = time.time() - start_time

# Evaluation
correct = 0
total = 0
net.eval()
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f"Training time: {training_time:.2f} seconds")
print(f"Final Accuracy: {accuracy:.2f}%")

100%|██████████| 170M/170M [00:06<00:00, 27.6MB/s]


Epoch 50, Loss: 0.1195
Epoch 100, Loss: 0.0803
Epoch 150, Loss: 0.0592
Epoch 200, Loss: 0.0582
Epoch 250, Loss: 0.0544
Epoch 300, Loss: 0.0668
Training time: 3565.78 seconds
Final Accuracy: 75.72%


In [ ]:
# Problem 1.b
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Data Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

# Extended CNN Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)  # NEW LAYER
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

        # After 4 conv+pool layers:
        # Input: 32x32 → conv1+pool → 16x16
        # → conv2+pool → 8x8
        # → conv3+pool → 4x4
        # → conv4+pool → 2x2
        # Feature map size = 256 * 2 * 2 = 1024
        self.fc1 = nn.Linear(256 * 2 * 2, 512)
        self.fc2 = nn.Linear(512, 10)  # 10 CIFAR-10 classes

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))  # NEW LAYER
        x = x.view(-1, 256 * 2 * 2)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

net = Net()

# Training Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training Loop
epochs = 300
start_time = time.time()

for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

training_time = time.time() - start_time

# Evaluation
correct = 0
total = 0
net.eval()
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f"Training time: {training_time:.2f} seconds")
print(f"Final Accuracy: {accuracy:.2f}%")

Epoch 50, Loss: 0.0497
Epoch 100, Loss: 0.0418
Epoch 150, Loss: 0.0402
Epoch 200, Loss: 0.0423
Epoch 250, Loss: 0.0398
Epoch 300, Loss: 0.0395
Training time: 3535.77 seconds
Final Accuracy: 73.99%


In [ ]:
# Problem 2.a
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Data Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# ResNet-10
class ResNet10(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet10, self).__init__()
        self.in_channels = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        # 10 blocks distributed across 3 stages
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride):
        strides = [stride] + [1]*(blocks-1)
        layers = []
        for s in strides:
            layers.append(block(self.in_channels, out_channels, s))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

net = ResNet10(ResidualBlock, [3,3,4])  # total 10 blocks

# Training Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training Loop
epochs = 300
start_time = time.time()

for epoch in range(epochs):
    net.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

training_time = time.time() - start_time

# Evaluation
net.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f"Training time: {training_time:.2f} seconds")
print(f"Final Accuracy: {accuracy:.2f}%")

Epoch 50, Loss: 0.0283
Epoch 100, Loss: 0.0173
Epoch 150, Loss: 0.0128
Epoch 200, Loss: 0.0115
Epoch 250, Loss: 0.0056
Epoch 300, Loss: 0.0038
Training time: 5117.36 seconds
Final Accuracy: 82.05%


In [ ]:
# Problem 2.b
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dropout_p=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(p=dropout_p) if dropout_p else nn.Identity()

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)  # Dropout applied here if enabled
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet10(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, dropout_p=None):
        super(ResNet10, self).__init__()
        self.in_channels = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1, dropout_p=dropout_p)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2, dropout_p=dropout_p)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2, dropout_p=dropout_p)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride, dropout_p):
        strides = [stride] + [1]*(blocks-1)
        layers = []
        for s in strides:
            layers.append(block(self.in_channels, out_channels, s, dropout_p))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

def train_model(net, optimizer, criterion, trainloader, testloader, epochs=300):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)
    start_time = time.time()

    for epoch in range(epochs):
        net.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        if (epoch+1) % 50 == 0:
            print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

    training_time = time.time() - start_time

    # Evaluation
    net.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return training_time, running_loss/len(trainloader), accuracy

criterion = nn.CrossEntropyLoss()

# Weight Decay
net_wd = ResNet10(ResidualBlock, [3,3,4])
optimizer_wd = optim.Adam(net_wd.parameters(), lr=0.001, weight_decay=0.001)
time_wd, loss_wd, acc_wd = train_model(net_wd, optimizer_wd, criterion, trainloader, testloader)

# Dropout
net_do = ResNet10(ResidualBlock, [3,3,4], dropout_p=0.3)
optimizer_do = optim.Adam(net_do.parameters(), lr=0.001)
time_do, loss_do, acc_do = train_model(net_do, optimizer_do, criterion, trainloader, testloader)

# Batch Normalization
net_bn = ResNet10(ResidualBlock, [3,3,4])
optimizer_bn = optim.Adam(net_bn.parameters(), lr=0.001)
time_bn, loss_bn, acc_bn = train_model(net_bn, optimizer_bn, criterion, trainloader, testloader)

print("Weight Decay:", time_wd, loss_wd, acc_wd)
print("Dropout:", time_do, loss_do, acc_do)
print("BatchNorm:", time_bn, loss_bn, acc_bn)


Epoch 50, Loss: 0.1492
Epoch 100, Loss: 0.1292
Epoch 150, Loss: 0.1184
Epoch 200, Loss: 0.1115
Epoch 250, Loss: 0.1086
Epoch 300, Loss: 0.1097
Epoch 50, Loss: 0.1706


Google colab's hosted runtime T4 can only allocate a given amount of runtime before stopping. Problem 2.b is incomplete due to this obstacle.